<a href="https://colab.research.google.com/github/smiledinisa/data_python_analysis/blob/master/pandas_006_DataAggregationAndGroupOperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
cd /content/drive/My Drive/examples

/content/drive/My Drive/examples


#CHAPTER 10 : Data Aggregation and Group Operations


groupby interface


In this chapter, we will learn how to:

Split a pandas object into pieces using one or more keys.(in the form of functions,arrays, DataFrame column names)  
对pandas对象进行split，用关键的key

Calculate group summary statistics, like count, mean, or standard deviation, or auser-defined function   
计算统计信息。


Apply within-group transformations or other manipulations, like normalization,
linear regression, rank, or subset selection  
应用内置group 转换操作。

Compute pivot tables and cross-tabulations  
计算旋转与正交表格


Perform quantile analysis and other statistical group analyses  
计算分位数分析信息。





## 10.1 GroupBy Mechanics


---
模型的GROUP ： split, apply, conbine.三部曲。


![链接文字](https://img-blog.csdnimg.cn/20200818174500445.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2dhb2N1aTg4Mw==,size_16,color_FFFFFF,t_70#pic_center)


- A list or array of values that is the same length as the axis being grouped


- A value indicating a column name in a DataFrame  


- A dict or Series giving a correspondence between the values on the axis being
grouped and the group names  

- A function to be invoked on the axis index or the individual labels in the index



In [3]:
# 看起来很抽象是吧，不用担心，慢慢往下看就可以了。

# 关键词： groupby and method:size

from pandas import DataFrame as DataFrame
from pandas import Series as Series 
import numpy as np
import pandas as pd


In [6]:
df = DataFrame({
    'key1':['a','a','b','b','a'],
    'key2' : ['one', 'two', 'one', 'two', 'one'],
    'data1' : np.random.randn(5),
    'data2' : np.random.randn(5)
})

df

,key1,key2,data1,data2
0,a,one,0.630660,-2.027697
1,a,two,0.339220,-0.150802
2,b,one,-0.959031,-0.180650
3,b,two,-0.288006,-0.436780
4,a,one,-0.036959,-0.890916


In [9]:
# we want to compute the mean of the data1 column using the labels from the key1.

grouped = df['data1'].groupby(df['key1']) 
grouped

In [10]:
# 可以看出，grouped 是一个groupby object。

grouped.mean()

key1
a    0.310974
b   -0.623518
Name: data1, dtype: float64

In [12]:
# if group by key1 & key2 ???

df['data1'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     one     0.296851
      two     0.339220
b     one    -0.959031
      two    -0.288006
Name: data1, dtype: float64

In [13]:
# df not only data1 ???

df.groupby([df['key1'],df['key2']]).mean()

data1     data2
key1 key2                    
a    one   0.296851 -1.459306
     two   0.339220 -0.150802
b    one  -0.959031 -0.180650
     two  -0.288006 -0.436780

In [15]:
# if we pass array


states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

x = df['data1'].groupby([states,years]).mean()
x

California  2005    0.339220
            2006   -0.959031
Ohio        2005    0.171327
            2006   -0.036959
Name: data1, dtype: float64

In [16]:
# 可以看出，传入array就是相当于新建了两个coloumns。
x.unstack()


,2005,2006
California,0.339220,-0.959031
Ohio,0.171327,-0.036959


In [20]:
df.groupby(['key1','key2']).size()  # 统计key1，key2 groupby的元素个数信息。

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [21]:
# Groupby object : 支持iteration， generating a sequence of 2-tuples containing the group name along with the chunk of data.

for name, group in df.groupby(['key1']):
  print(name)
  print(group)

a
  key1 key2     data1     data2
0    a  one  0.630660 -2.027697
1    a  two  0.339220 -0.150802
4    a  one -0.036959 -0.890916
b
  key1 key2     data1    data2
2    b  one -0.959031 -0.18065
3    b  two -0.288006 -0.43678


In [23]:
# 如果是多个key

for (k1,k2), group in df.groupby(['key1','key2']):
  print((k1,k2))
  print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.630660 -2.027697
4    a  one -0.036959 -0.890916
('a', 'two')
  key1 key2    data1     data2
1    a  two  0.33922 -0.150802
('b', 'one')
  key1 key2     data1    data2
2    b  one -0.959031 -0.18065
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.288006 -0.43678


In [26]:
# iteration 特点的对象可以转换为list和dic对象。这样用key来访问即可。
pieces = dict(list(df.groupby('key1')))
print(pieces['a'])
print(pieces['b'])

  key1 key2     data1     data2
0    a  one  0.630660 -2.027697
1    a  two  0.339220 -0.150802
4    a  one -0.036959 -0.890916
  key1 key2     data1    data2
2    b  one -0.959031 -0.18065
3    b  two -0.288006 -0.43678


In [27]:
# groupby 默认是按 axis=0 来进行的。
# 但如果我们要按列的特点进行groupby???

grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
  print(dtype)
  print(group)

float64
      data1     data2
0  0.630660 -2.027697
1  0.339220 -0.150802
2 -0.959031 -0.180650
3 -0.288006 -0.436780
4 -0.036959 -0.890916
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [ ]:
# 用groupby对象的列名或者列名的列表，来访问和统计局部数据将非常方便。

In [28]:
df.groupby('key1')['data1']

In [29]:
df.groupby('key1')[['data1']]

In [30]:
# 可以看出，如果是单个的列名，则返回一个SeriesGroupby.
# 如果是一个列名的列表，则返回一个DataFrameGroupby对象。

# 效果跟：
# df['data1'].groupby(df['key1'])
# df[['data1']].groupby(df['key1'])
# 一样的。？

In [31]:
df.groupby(['key1','key2'])[['data2']].mean() # return DataFrame.

data2
key1 key2          
a    one  -1.459306
     two  -0.150802
b    one  -0.180650
     two  -0.436780

In [32]:
df.groupby(['key1','key2'])['data2'].mean()  # only a Series.

key1  key2
a     one    -1.459306
      two    -0.150802
b     one    -0.180650
      two    -0.436780
Name: data2, dtype: float64

### Grouping with Dicts and Series


In [33]:
# 用字典和series来传入

people = DataFrame(np.random.randn(5,5), columns=['a','b','c','d','e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.465804,0.773071,-0.440803,0.393922,0.674543
Steve,-1.455166,-1.089501,1.125910,0.568999,-0.799203
Wes,-0.862717,NaN,NaN,-1.514971,2.063249
Jim,-1.377478,0.201512,-2.354783,-0.415470,0.780181
Travis,-0.071163,1.016896,0.447399,1.672246,0.883602


In [36]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}

In [38]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [39]:
# 用字典来groupby 
by_column = people.groupby(mapping, axis=1)
by_column.sum()



,blue,red
Joe,-0.046881,0.981811
Steve,1.694909,-3.343870
Wes,-1.514971,1.200532
Jim,-2.770253,-0.395785
Travis,2.119646,1.829335


In [42]:
series_column = people.groupby(map_series, axis=1).sum()
series_column

,blue,red
Joe,-0.046881,0.981811
Steve,1.694909,-3.343870
Wes,-1.514971,1.200532
Jim,-2.770253,-0.395785
Travis,2.119646,1.829335


### Grouping with Functions

In [43]:
# 类似与字典，这种方法更加普世一些。
# 我们用名字的长度来进行groupby 

people.groupby(len).sum() #默认调用index，返回的数值用作新的index。

,a,b,c,d,e
3,-2.705998,0.974583,-2.795586,-1.536519,3.517973
5,-1.455166,-1.089501,1.125910,0.568999,-0.799203
6,-0.071163,1.016896,0.447399,1.672246,0.883602


In [45]:
# 各种方法联合使用。
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.862717  0.773071 -0.440803 -1.514971  0.674543
  two -1.377478  0.201512 -2.354783 -0.415470  0.780181
5 one -1.455166 -1.089501  1.125910  0.568999 -0.799203
6 two -0.071163  1.016896  0.447399  1.672246  0.883602

### Grouping by Index Levels

In [46]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                    [1, 3, 5, 1, 3]],
                    names=['cty', 'tenor'])

In [48]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -2.185981  0.815238  1.470625  1.139690 -1.000751
1      0.012733  0.397970 -0.826492  1.129101 -0.624884
2      1.051360  0.131418  0.192288 -0.294671 -0.197234
3     -0.847890 -1.775263 -1.091356 -0.259385  0.078173

In [49]:
hier_df.groupby(level='cty', axis=1).mean()

cty,JP,US
0,0.069470,0.033294
1,0.252109,-0.138597
2,-0.245953,0.458355
3,-0.090606,-1.238170


## 10.2 Data Aggregation

![链接文字](https://img-blog.csdnimg.cn/20200819110058967.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2dhb2N1aTg4Mw==,size_16,color_FFFFFF,t_70#pic_center)

In [50]:
df

,key1,key2,data1,data2
0,a,one,0.630660,-2.027697
1,a,two,0.339220,-0.150802
2,b,one,-0.959031,-0.180650
3,b,two,-0.288006,-0.436780
4,a,one,-0.036959,-0.890916


In [52]:
# aggregation 数据聚合。大致意思就是从array到标量的转换，比如sum就是可以从一组数得到一个标量（总和）

# 分为数：quantile 

grouped = df.groupby('key1')
grouped['data1'].quantile(0.5) #计算每个group 0.5分位数。

key1
a    0.339220
b   -0.623518
Name: data1, dtype: float64

In [53]:
# 也可以用自己的自定义函数。custom function.
def peaktopeak(arr):
  return arr.max()-arr.min()

grouped.agg(peaktopeak)  # 注意agg或者aggregate

,data1,data2
key1,,
a,0.667620,1.876895
b,0.671026,0.256130


In [54]:
# describe()函数虽然不是aggregate 函数，但是却可以直接使用。

grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0  0.310974  0.334705  ... -0.890916 -0.520859 -0.150802
b      2.0 -0.623518  0.474487  ... -0.308715 -0.244682 -0.180650

[2 rows x 16 columns]

### Column-Wise and Multiple Function Application

In [55]:
# 本节解决如下问题： 每一列应用相同的多个统计函数。
# 多个列应用多个统计函数。
# 多个列分别应用不同的统计函数。

tips = pd.read_csv('tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [58]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [59]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct

In [60]:
# 一行，多个group 应用一个函数。
grouped_pct.aggregate('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [62]:
# 一列，多个group，应用多个函数。
grouped_pct.agg(['mean', 'std', peaktopeak]) # 注意带中括号。


mean       std  peaktopeak
day  smoker                                
Fri  No      0.151650  0.028123    0.067349
     Yes     0.174783  0.051293    0.159925
Sat  No      0.158048  0.039767    0.235193
     Yes     0.147906  0.061375    0.290095
Sun  No      0.160113  0.042347    0.193226
     Yes     0.187250  0.154134    0.644685
Thur No      0.160298  0.038774    0.193350
     Yes     0.163863  0.039389    0.151240

In [63]:
# 如果我们要顺便改掉结果名呢？？？

# 传递一个元组的列表。
# 元组的第一个元素就是新的列名，第二个参数就是函数名。

grouped_pct.aggregate([('fuck1','mean'), ('fuck2','std'), ('fuck3', peaktopeak)])

fuck1     fuck2     fuck3
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [66]:
# 如果是个DataFrame 呢。也就多个列。

functions =  ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions) # 对两列使用多个函数。
result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [68]:
# 给人的感觉就是将两个单列结果的进行了链接。
result['tip_pct']


count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [69]:
result['total_bill']

count       mean    max
day  smoker                         
Fri  No          4  18.420000  22.75
     Yes        15  16.813333  40.17
Sat  No         45  19.661778  48.33
     Yes        42  21.276667  50.81
Sun  No         57  20.506667  48.17
     Yes        19  24.120000  45.35
Thur No         45  17.113111  41.19
     Yes        17  19.190588  43.11

In [73]:
reslut2 = pd.concat([result['tip_pct'], result['total_bill']], axis=1)
reslut2

count      mean       max  count       mean    max
day  smoker                                                    
Fri  No          4  0.151650  0.187735      4  18.420000  22.75
     Yes        15  0.174783  0.263480     15  16.813333  40.17
Sat  No         45  0.158048  0.291990     45  19.661778  48.33
     Yes        42  0.147906  0.325733     42  21.276667  50.81
Sun  No         57  0.160113  0.252672     57  20.506667  48.17
     Yes        19  0.187250  0.710345     19  24.120000  45.35
Thur No         45  0.160298  0.266312     45  17.113111  41.19
     Yes        17  0.163863  0.241255     17  19.190588  43.11

In [75]:
# 如果要对不同的列，使用不同的函数该如何处理呢？

grouped.size()

day   smoker
Fri   No         4
      Yes       15
Sat   No        45
      Yes       42
Sun   No        57
      Yes       19
Thur  No        45
      Yes       17
dtype: int64

In [76]:
# 对不同的列使用不同列使用不同的单个函数。 tip列使用 max函数。对size列使用sum函数。
grouped.agg({'tip':'max', 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [77]:
# 对不同的列使用不同的函数组。
grouped.aggregate({'tip':['max','min','mean'], 'size':['sum','std']})

tip                 size          
               max   min      mean  sum       std
day  smoker                                      
Fri  No       3.50  1.50  2.812500    9  0.500000
     Yes      4.73  1.00  2.714000   31  0.593617
Sat  No       9.00  1.00  3.102889  115  0.784960
     Yes     10.00  1.00  2.875476  104  0.862161
Sun  No       6.00  1.01  3.167895  167  1.032674
     Yes      6.50  1.50  3.516842   49  0.901591
Thur No       6.70  1.25  2.673778  112  1.179796
     Yes      5.00  2.00  3.030000   40  0.701888

In [79]:
# 顺便帮其改名。
grouped.aggregate({'tip':[('fuck1','max'),('fuck2','min'),('fuck3','mean')], 'size':['sum','std']})

tip                 size          
             fuck1 fuck2     fuck3  sum       std
day  smoker                                      
Fri  No       3.50  1.50  2.812500    9  0.500000
     Yes      4.73  1.00  2.714000   31  0.593617
Sat  No       9.00  1.00  3.102889  115  0.784960
     Yes     10.00  1.00  2.875476  104  0.862161
Sun  No       6.00  1.01  3.167895  167  1.032674
     Yes      6.50  1.50  3.516842   49  0.901591
Thur No       6.70  1.25  2.673778  112  1.179796
     Yes      5.00  2.00  3.030000   40  0.701888

### Returning Aggregated Data Without Row Indexes

## 10.3 Apply: General split-apply-combine

### Suppressing the Group Keys

### Quantile and Bucket Analysis

### Example: Filling Missing Values with Group-Specific Values

### Example: Random Sampling and Permutation

### Example: Group Weighted Average and Correlation



### Example: Group-Wise Linear Regression

## 10.4 Pivot Tables and Cross-Tabulation

### Cross-Tabulations: Crosstab